In [1]:
import numpy as np
from copy import copy
import random
import time
import math
import sympy as sy
import itertools
from fractions import Fraction
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display,display_latex

In [2]:
url='https://www.janestreet.com/puzzles/robot-archery-index/'
res = requests.get(url)
soup = BeautifulSoup(res.content, 'html.parser')
y =[text for text in soup.body.stripped_strings]
#display([[i,j] for i,j in enumerate(y)])
display(Markdown("### "+y[8]+"\n\n"+str("\n".join(y[10:76]))))

### December 2021 : Puzzle

After a grueling year filled with a wide variety of
robot
sporting
events
, we have arrived at the final event of the year:
Robot Archery
. Four robots have qualified for this year’s finals, and have been seeded in the following order:
Robot
Seed
Aaron
1
Barron
2
Caren
3
Darrin
4
The robots will take turns shooting arrows at a target
1
, starting with Aaron and proceeding in order by seed. When it is a given robot’s turn, they shoot a single arrow. If it is closer to the center of the target than
all
previous arrows by all players, that robot remains in the tournament, going to the back of the queue to await their next turn. Otherwise that robot is eliminated immediately. The last robot remaining in the queue is the winner.
For example, here is how
last year’s
finals went, in which
Caren
was the winner. (Oddly enough it involved the same robots in the same seeding.)
Turn
Robot
Distance
1
Aaron
10nm
2
Barron
8nm
3
Caren
7nm
4
Darrin
1km
5
Aaron
9nm
6
Barron
2nm
7
Caren
1nm
8
Barron
1Ym
2
To ten decimal places, what is the probability that
Darrin
will be this year’s winner?
(Or, if you want to send in the
exact answer
, that’s fine too!)
[1] Each robot is equally skilled. Which is to say: for any region
R
on the target with nonzero area, the robots all have the same positive probability of landing an arrow within
R
on any given shot.
[2] It’s a large target.

In [3]:
# build out the probability tree
def tree(p,stopper):
    game = [(np.ones(p),Fraction(1,1),1,1)]  # live players,probability, number of shots, next player
    result = []
    
    while len(game) >0:
        (players,prob,num,x) = game.pop()
        if np.sum(players) ==1:
            result.append([np.argmax(players)+1,prob])
        elif num >stopper:
            result.append([0,prob])
        else:
            num +=1
            pass_prob = prob * Fraction(1,num)
            fail_prob = prob * Fraction(num-1,num)
            fail_players = copy(players)
            fail_players[x] = 0
            nextp = (x +1) % p
            while players[nextp] ==0:
                nextp = (nextp +1) % p
            game.append([players,pass_prob,num,nextp])
            game.append([fail_players,fail_prob,num,nextp])
    print(len(result),"total paths")
    return [(n,float(np.sum([j for i,j in result if i==n])))for n in range(p+1)]


In [4]:
output = tree(4,18)
for i in range(5):
    print("Player {} has a {:.10f} probability".format(output[i][0],output[i][1]))

988 total paths
Player 0 has a 0.0000000000 probability
Player 1 has a 0.3711532348 probability
Player 2 has a 0.2421885526 probability
Player 3 has a 0.2032205617 probability
Player 4 has a 0.1834376509 probability


In [5]:
url='https://www.janestreet.com/puzzles/robot-archery-solution/'
res = requests.get(url)
soup = BeautifulSoup(res.content, 'html.parser')
y =[text for text in soup.body.stripped_strings]
#display([[i,j] for i,j in enumerate(y)])
display(Markdown("### "+y[7]+"\n\n"+str("\n".join(y[10:219]))))

### Robot Archery

[Above is a Mathematica-derived solution to the systems of equations
described in the 4-player tournament described below]
Without loss of generality we can assume an arrow’s distance to the center of the target is U[0,1] distributed. Let
P
j,k
(
x
) denote the probability that Player
j
out of
k
in the
current
line will eventually win the game, given that the current best dart is at distance
x
.
In a 2-player archery tournament, we would have
P
1,2
(
x
) +
P
2,2
(
x
) = 1,
P
1,2
(0) = 0, and
P
1,2
(
x
) = ∫
0
x
P
2,2
(
u
) d
u
. Taking derivatives of both sides in that last equation and substituing gives
P
‘
1,2
(
x
) = −
P
1,2
(x). Solving gives
P
1,2
(
x
) = 1−e
−
x
.
In the 3-player version of the game, we get the following system of equations:
P
1,3
(
x
) = ∫
0
x
P
3,3
(
u
) d
u
P
2,3
(
x
) = (1 -
x
)
P
1,2
(
x
) + ∫
0
x
P
1,3
(
u
) d
u
P
1,3
(
x
) +
P
2,3
(
x
) +
P
3,3
(
x
) = 1
P
1,3
(0) =
P
2,3
(0) = 0.
And for the 4-player version we have the following:
P
1,4
(
x
) = ∫
0
x
P
4,4
(
u
) d
u
P
2,4
(
x
) = (1 -
x
)
P
1,3
(
x
) + ∫
0
x
P
1,4
(
u
) d
u
P
3,4
(
x
) = (1 -
x
)
P
2,3
(
x
) + ∫
0
x
P
2,4
(
u
) d
u
P
1,4
(
x
) +
P
2,4
(
x
) +
P
3,4
(
x
) +
P
4,4
(
x
) = 1
P
1,4
(0) =
P
2,4
(0) =
P
3,4
(0) = 0.
In particular we have
P
4,4
(1) = (-5/4)*(cos(1) + sin(1)) + (1/2)*
e
-1
+ (
e
-1/2
)*(cos(sqrt(3)/2) + 5/sqrt(3)*sin(sqrt(3)/2)), or approximately 0.18343765086.
Congrats to this month’s solvers!

In [6]:
#exact solution. 4 player game doesn't simplify to the 3 player game on hit so gets tricky. 2 player game is tractable.
import sympy as sy
n = sy.symbols("n")

print("For the 2 player game the probability of second player winning is ")

form = sy.Sum((1-1/(2*n+1))*1/(sy.factorial(2*n)),(n,1,sy.oo))
print(form)
print(form.doit())


For the 2 player game the probability of second player winning is 
Sum((1 - 1/(2*n + 1))/factorial(2*n), (n, 1, oo))
-sinh(1) + cosh(1)


In [7]:
# simulate to check 
def game(players=4):
    live = np.ones(players)
    distance = np.sum(np.random.rand(2)**2)**0.5   
    player = 1
    while sum(live) > 1:
        if live[player] ==1:
            shot = np.sum(np.random.rand(2)**2)**0.5   
            if shot > distance:
                live[player] = 0
            else:
                distance = shot
        player = (player + 1 ) % players
        while live[player] ==0:
            player = (player + 1 ) % players
    return (live)    

In [8]:
start = time.time()
N= 10**5
for i in range(3):
    print(np.sum([game(4) for i in range(N)],axis=0)/N)
print("Took",time.time()-start)

[0.37055 0.24212 0.2032  0.18413]
[0.37265 0.24353 0.2011  0.18272]
[0.37304 0.23981 0.20306 0.18409]
Took 16.396159410476685


In [9]:
# Try to work through the solution given
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


#set up symbols
x,u= sy.symbols('x u')
# set up functions
probs = sum([["P_"+str(j+1)+str(i) for j in range(i)] for i in range(2,5) ],[])


for i in probs:
    locals()[i] =sy.Function(i) 

# 2 player case,starting conditions
eq1 = sy.Eq(P_12(x) + P_22(x),1)
eq2 = sy.Eq(P_12(0),0)
eq3 = sy.Eq(P_12(x),sy.Integral(P_22(u),(u,0,x)))
display(Markdown("***Initial Conditions***"))
display(eq1,eq2,eq3)

# differentiate eq3 and then solve the ODE including eq2 as an initial condition
eq4 = sy.Eq(sy.diff(eq3.lhs,x),sy.diff(eq3.rhs,x))
eq5 = sy.Eq(P_12(x),sy.solve([eq1,eq2,eq4])[0][P_12(x)])
display(eq5)
eq6 = sy.dsolve(eq5,ics={P_12(0):0})
display(Markdown("***Solving gives***"))
display(eq6)

***Initial Conditions***

Eq(P_12(x) + P_22(x), 1)

Eq(P_12(0), 0)

Eq(P_12(x), Integral(P_22(u), (u, 0, x)))

Eq(P_12(x), 1 - Derivative(P_12(x), x))

***Solving gives***

Eq(P_12(x), 1 - exp(-x))

In [10]:
display(Markdown("***Expand to 3 player***"))
eq7 = sy.Eq(P_13(x),sy.Integral(P_33(u),(u,0,x)))
eq8 = sy.Eq(P_23(x),(1-x)*P_12(x)+sy.Integral(P_13(u),(u,0,x)))
eq9 = sy.Eq(P_13(x) + P_23(x) + P_33(x),1)
eq10 = sy.Eq(P_13(0),0)
eq11 = sy.Eq(P_23(0),0)
display(eq7,eq8,eq9,eq10,eq11)

***Expand to 3 player***

Eq(P_13(x), Integral(P_33(u), (u, 0, x)))

Eq(P_23(x), (1 - x)*P_12(x) + Integral(P_13(u), (u, 0, x)))

Eq(P_13(x) + P_23(x) + P_33(x), 1)

Eq(P_13(0), 0)

Eq(P_23(0), 0)

In [11]:
display(Markdown("***Manipulate***"))
eq12 = sy.Eq(sy.diff(eq7.rhs,x),sy.diff(eq7.lhs,x))
eq13 = sy.Eq(P_13(x),sy.solve([eq6,eq8,eq9,eq12])[0][P_13(x)])
eq14 = sy.Eq(sy.diff(eq13.lhs,x),sy.diff(eq13.rhs,x))
display(eq12,eq13,eq14)

display(Markdown("\n***Solve eq14***"))
eq15 = sy.dsolve(eq14,ics={P_13(0):0,P_13(x).diff(x).subs(x,0):1})
display(eq15)
display(eq15.subs(x,1).evalf())

***Manipulate***

Eq(P_33(x), Derivative(P_13(x), x))

Eq(P_13(x), x - x*exp(-x) - Derivative(P_13(x), x) - Integral(P_13(u), (u, 0, x)) + exp(-x))

Eq(Derivative(P_13(x), x), x*exp(-x) - P_13(x) - Derivative(P_13(x), (x, 2)) + 1 - 2*exp(-x))


***Solve eq14***

Eq(P_13(x), (x - 1)*exp(-x) + 1 - 2*sqrt(3)*exp(-x/2)*sin(sqrt(3)*x/2)/3)

Eq(P_13(1), 0.466492804885307)

In [12]:
display(Markdown("***From***"))
display(eq12)
display(Markdown("***we get***"))
eq16 =sy.Eq(P_33(x),eq15.rhs.diff(x))
display(eq16)
display(eq16.subs(x,1).evalf())

***From***

Eq(P_33(x), Derivative(P_13(x), x))

***we get***

Eq(P_33(x), -(x - 1)*exp(-x) + exp(-x) + sqrt(3)*exp(-x/2)*sin(sqrt(3)*x/2)/3 - exp(-x/2)*cos(sqrt(3)*x/2))

Eq(P_33(1), 0.241686482894434)

In [13]:
display(Markdown("***So***"))
eq17 =sy.Eq(P_23(x),1-eq16.rhs-eq15.rhs)
display(eq17)
display(eq17.subs(x,1).evalf())

***So***

Eq(P_23(x), -exp(-x) + sqrt(3)*exp(-x/2)*sin(sqrt(3)*x/2)/3 + exp(-x/2)*cos(sqrt(3)*x/2))

Eq(P_23(1), 0.291820712220259)

In [14]:
#Check vs tree
output = tree(3,18)
for i in range(4):
    print("Player {} has a {:.10f} probability".format(output[i][0],output[i][1]))

172 total paths
Player 0 has a 0.0000000000 probability
Player 1 has a 0.4664928049 probability
Player 2 has a 0.2918207122 probability
Player 3 has a 0.2416864829 probability


In [15]:
eq18 = sy.Eq(P_14(x),sy.Integral(P_44(u),(u,0,x)))
eq19 = sy.Eq(P_24(x),(1-x)*P_13(x)+sy.Integral(P_14(u),(u,0,x)))

eq20 = sy.Eq(P_34(x),(1-x)*P_23(x)+sy.Integral(P_24(u),(u,0,x)))
eq21 = sy.Eq(P_14(x) + P_24(x) + P_34(x) + P_44(x),1)
eq22 = sy.Eq(P_14(0),0)
eq23 = sy.Eq(P_24(0),0)
eq24 = sy.Eq(P_34(0),0)
display(Markdown("***Using***"))
display(eq15,eq17)
display(Markdown("***And on to 4 player***"))
display(eq18,eq19,eq20,eq21,eq22,eq23,eq24)

***Using***

Eq(P_13(x), (x - 1)*exp(-x) + 1 - 2*sqrt(3)*exp(-x/2)*sin(sqrt(3)*x/2)/3)

Eq(P_23(x), -exp(-x) + sqrt(3)*exp(-x/2)*sin(sqrt(3)*x/2)/3 + exp(-x/2)*cos(sqrt(3)*x/2))

***And on to 4 player***

Eq(P_14(x), Integral(P_44(u), (u, 0, x)))

...

Eq(P_34(x), (1 - x)*P_23(x) + Integral(P_24(u), (u, 0, x)))

Eq(P_14(x) + P_24(x) + P_34(x) + P_44(x), 1)

Eq(P_14(0), 0)

Eq(P_24(0), 0)

Eq(P_34(0), 0)

In [16]:
display(Markdown("***Manipulate***"))
eq25 = sy.Eq(sy.diff(eq18.rhs,x),sy.diff(eq18.lhs,x))
eq26 = sy.Eq(sy.diff(eq19.rhs,x),sy.diff(eq19.lhs,x))
eq27 = sy.Eq(sy.diff(eq20.rhs,x),sy.diff(eq20.lhs,x))
display(eq25,eq26,eq27)

***Manipulate***

Eq(P_44(x), Derivative(P_14(x), x))

Eq((1 - x)*Derivative(P_13(x), x) - P_13(x) + P_14(x), Derivative(P_24(x), x))

Eq((1 - x)*Derivative(P_23(x), x) - P_23(x) + P_24(x), Derivative(P_34(x), x))

In [17]:
#sy.solve([eq25,eq19,eq20,eq21,eq15,eq17])